In [252]:
import math
import random

In [253]:
def exponencial(lde,t):
    return -((1/lde)*math.log(random.random()))

In [254]:
def poisson(ldpp,t):
    return t-((1/ldpp)*math.log(random.random()))

In [255]:
def getFree(td):
    for i in range(len(td)):
        if(td[i] == float('Inf')):
            return i #el servidor está libre
    return -1 #ningún servidor esta libre

In [256]:
#llega evento al sistema
def caso1(t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T):
    t = ta
    Na+=1
    n+=1
    Tt = poisson(ldpp,t)
    ta = Tt
    #revisar si alguno está libre
    index = getFree(td)
    if(index != -1):
        Inf[index] = Na-1 #almaceno ID (Na=1 pero array inicia en 0)
        td[index] = t + exponencial(lde,t) #se almacena el tiempo en que saldra este evento
        lastID = lastID+1
    A.append(t)
    D.append(float('Inf')) #se hace el espacio del evento
    return t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T

In [257]:
#un evento en el sistema a finalizado
def caso2(t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T):
    t = min(td)
    index = td.index(min(td)) #obtengo el servidor que lo está ejecutando
    n-=1
    Nd+=1
    #print("--------------------")
    #print("td: ", td)
    #print("index: ",index)
    #print("Inf: ",Inf[index])
    #print("D: ", D)
    
    D[Inf[index]] = t
    C[index] = C[index]+1 #atendio otro evento
    
    #no hay mas eventos (IDLE)
    if(n == 0):
        Inf[index] =  -1 #no hay id de evento
        td[index] = float('Inf') #no hay tiempo de salida
    else:
        Y = exponencial(lde,t)
        #td = t + Y
        Inf[index] = lastID+1
        td[index] = t + Y
        lastID = lastID+1
    return t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T

In [258]:
def caso3(t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T):
    t = min(td)
    index = td.index(min(td)) #obtengo el servidor que lo está ejecutando
    n-=1
    Nd+=1
    D[Inf[index]] = t
    
    if(n > 0):
        Y = exponencial(lde,t)
        #td = t + Y
        C[index] = C[index] + 1 #atiendo otro evento
        Inf[index] = lastID + 1
        td[index] = t+Y
        lastID = lastID+1
    #else:
        #no hay eventos para el servidor
        #Tp[index] = max(t-T,0)
        #Inf[index] = -1
        #td[index] = float('Inf')
    return t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T

In [259]:
def caso4(t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T):
    t_old = t
    t = min(td)
    if(t != float('Inf')):
        index = td.index(min(td))
        Nd += 1
        D[Inf[index]] = t
    
        #no hay eventos en el servidor
        Tp[index] = max(t_old - T,0)
        Inf[index] = -1
        td[index] = float('Inf')
    else:
        B = 1
    return t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,B,T

In [260]:
def selector(t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T):
    B = 0
    if(ta < min(td) and ta <= T):
        #evento es llegada y sistema con tiempo
        #caso 1
        t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T = caso1(t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T)
    elif(min(td)<ta and min(td) <= T):
        #evento es salida y el sistema con tiempo
        #caso 2
        t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T = caso2(t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T)
    elif(min(ta,min(td)) > T and n > 0):
        #proximo evento luego de cierre y hay solicitudes
        #caso 3
        t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T = caso3(t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T)
    else:
        #proximo evento luego de cierre y no hay solicitudes
        #caso 4
        t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,B,T = caso4(t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T)
    return t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,B,T

In [261]:
def Sistema(T1, ldpp1, lde1, servidores):
    #Sistema
    #parametros para simulacion
    T = T1
    lde = lde1
    ldpp = ldpp1

    #parametros del sistema
    t = 0 #tiempo actual
    Na = 0 #eventos llegados
    Nd = 0 #eventos atendidos
    n = 0 #eventos actuales
    ta = poisson(ldpp, t) #tiempo llegada siguiente
    td = [] #tiempo de salida del evento atendido por el servidor i (cada servidor)
    Tp = [] #tiempo extra de cada servidor
    A = [] #tiempo de llegada del cliente i
    D = [] #tiempo de salida del cliente i
    Inf = [] #ID del evento atendido por el servidor i
    C = [] #cuantos eventos ha atendido cada servidor
    
    B = 0
    lastID = -1
    
    for i in range(servidores):
        td.append(float('Inf'))
        Tp.append(0)
        A.append(0)
        D.append(float('Inf'))
        Inf.append(-1)
        C.append(0)
    
    while(t<T and B == 0):
        t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,B,T = selector(t,Na,Nd,n,ta,td,A,D,Inf,C,Tp,ldpp,lde,lastID,T) 
    print("Solicitudes ingresadas",Na)
    print("Solicitudes egresadas", Nd)




In [262]:
Sistema(1,2400/60,100,4)

index:  0
Inf:  0
D:  [inf, inf, inf, inf, inf]
index:  0
Inf:  1
D:  [0.010838150324920704, inf, inf, inf, inf, inf]
index:  0
Inf:  2
D:  [0.010838150324920704, 0.023072921944224722, inf, inf, inf, inf, inf]
index:  0
Inf:  3
D:  [0.010838150324920704, 0.023072921944224722, 0.035521957614202515, inf, inf, inf, inf, inf]
index:  0
Inf:  4
D:  [0.010838150324920704, 0.023072921944224722, 0.035521957614202515, 0.0485784398645548, inf, inf, inf, inf, inf]
index:  0
Inf:  5
D:  [0.010838150324920704, 0.023072921944224722, 0.035521957614202515, 0.0485784398645548, 0.21550862731926318, inf, inf, inf, inf, inf]
index:  0
Inf:  6
D:  [0.010838150324920704, 0.023072921944224722, 0.035521957614202515, 0.0485784398645548, 0.21550862731926318, 0.2350518621499374, inf, inf, inf, inf, inf]
index:  0
Inf:  7
D:  [0.010838150324920704, 0.023072921944224722, 0.035521957614202515, 0.0485784398645548, 0.21550862731926318, 0.2350518621499374, 0.2504392033977371, inf, inf, inf, inf, inf]
index:  0
Inf:  8

IndexError: list assignment index out of range